In [37]:
# pip install censusgeocode

In [22]:
import pandas as pd
import numpy as np
import censusgeocode as cg
import time
from datetime import datetime

import geopandas as gpd
import os

### Setting

1. Clinic/Center - Amputee: 261QA0900X
2. Orthotist: 222Z00000X
3. Prosthetist: 224P00000X
4. Prosthetic/Orthotic Supplier:335E00000X
<br>/////////////////////
5. Prosthetics Case Management: 1744P3200X
6. Orthotic Fitter: 225000000X
7. Pedorthist: 224L00000X

In [23]:
keep_col = ['NPI','Entity Type Code','Provider Organization Name (Legal Business Name)',
            'Provider First Line Business Practice Location Address',
            'Provider Business Practice Location Address City Name',
            'Provider Business Practice Location Address State Name',
            'Provider Business Practice Location Address Postal Code',
            'Provider Business Practice Location Address Telephone Number']
taxon_codes = ['Healthcare Provider Taxonomy Code_' + str(i+1) for i in range(15)]
taxonswitch_codes = ['Healthcare Provider Primary Taxonomy Switch_' + str(i+1) for i in range(15)]
keep_col += taxon_codes
keep_col += taxonswitch_codes

community_pharm = ['261QA0900X', '222Z00000X', '224P00000X','335E00000X']
npi_csv = 'npidata_pfile_20050523-20230212.csv'

end_str = [' STE', ' SUITE', ' BLDG', ' TOWER', ', #', ' UNIT',
           ' APT', ' BUILDING',',', '#']

In [24]:
def sub_rows(data):
    ec = data['Entity Type Code'] == "2"
    st = data['Provider Business Practice Location Address State Name'] == 'CA'
    ta = data[taxon_codes].isin(community_pharm).any(axis=1)
    #ac = data['NPI Deactivation Reason Code'].isna()
    all_together = ec & st & ta  #& ac 
    sub = data[all_together]
    return sub

def csv_chunks(file,chunk_size,keep_cols,row_sub):
    header_fields = list(pd.read_csv(npi_csv, nrows=1))
    header_locs = [header_fields.index(i) for i in keep_cols]
    skip = 1
    it_n = 0
    sub_n = 0
    ret_chunk = chunk_size
    fin_li_dat = []
    while ret_chunk == chunk_size:
        file_chunk = pd.read_csv(file, usecols=header_locs, skiprows=skip, 
                     nrows=chunk_size, names=header_fields, dtype='str')
        sub_dat = row_sub(file_chunk)
        fin_li_dat.append( sub_dat.copy() )
        skip += chunk_size
        it_n += 1
        sub_n += sub_dat.shape[0]
        print(f'Grabbed iter {it_n} total sub n so far {sub_n}')
        ret_chunk = file_chunk.shape[0]
    fin_dat = pd.concat(fin_li_dat, axis=0)
    return fin_dat

def clean_add(address):
    add_new = address.upper()
    for su in end_str:
        sf = address.find(su)
        if sf > -1:
            add_new = add_new[0:sf]
    add_new = add_new.replace('.','')
    add_new = add_new.strip()
    return add_new

### Run

In [25]:
# Takes about 3 minutes
print( datetime.now() )
pharm_tx = csv_chunks(npi_csv, chunk_size=1000000, keep_cols=keep_col, row_sub=sub_rows)
print( datetime.now() )

2023-04-18 13:58:37.960362
Grabbed iter 1 total sub n so far 67
Grabbed iter 2 total sub n so far 191
Grabbed iter 3 total sub n so far 345
Grabbed iter 4 total sub n so far 448
Grabbed iter 5 total sub n so far 529
Grabbed iter 6 total sub n so far 664
Grabbed iter 7 total sub n so far 761
Grabbed iter 8 total sub n so far 846
2023-04-18 14:00:48.704829


## Data Cleaning

In [26]:
ph_tx = pharm_tx

In [27]:
ph_tx.shape

(846, 38)

In [28]:
ph_tx['Provider Business Practice Location Address Postal Code'] = ph_tx['Provider Business Practice Location Address Postal Code'].str[0:5]
ph_tx['Zip5'] = ph_tx['Provider Business Practice Location Address Postal Code'].str[0:5]
ph_tx['Address'] = ph_tx['Provider First Line Business Practice Location Address'].apply(clean_add)
ph_tx.rename(columns={'Provider Business Practice Location Address City Name':'City',
                      'Provider Business Practice Location Address State Name':'State2'}
             ,inplace=True)
ph_tx = ph_tx[-ph_tx['Provider Organization Name (Legal Business Name)'].str.contains("NORDSTROM")].reset_index(drop = True)

In [29]:
newc = []
for n in range(len(ph_tx)):
    for i in range(15):
        if ph_tx.iloc[n]['Healthcare Provider Primary Taxonomy Switch_' + str(i+1)] == 'Y':
            v = ph_tx.iloc[n]['Healthcare Provider Taxonomy Code_' + str(i+1)]
    newc.append(v)

In [30]:
ph_tx = ph_tx.drop(columns=taxon_codes+taxonswitch_codes).reset_index(drop=True)

In [31]:
ph_tx['taxonomy'] = newc

In [32]:
# ph_tx = ph_tx[ph_tx['taxonomy'].isin(community_pharm)]

In [33]:
ph_tx2 = ph_tx[ph_tx['taxonomy'].str.contains("261QA0900X")]

In [34]:
ph_tx = ph_tx[ph_tx['Provider Organization Name (Legal Business Name)'].str.contains("ORTHO|PROS|P&O|LIMB")].reset_index(drop = True)

In [35]:
ph_tx.shape

(391, 11)

In [36]:
ph_tx_3 = pd.concat([ph_tx, ph_tx2]).drop_duplicates()

In [37]:
ph_tx_3.shape

(395, 11)

In [39]:
ph_tx_3.to_excel('list_caliclinic.xlsx',index=False)

## version1:

In [8]:
# ph_tx = ph_tx.drop(columns=taxon_codes+taxonswitch_codes).reset_index(drop=True)

In [9]:
# ph_tx = ph_tx[ph_tx['Provider Organization Name (Legal Business Name)'].str.contains("ORTHO|PROS|P&O|LIMB")]

## version2:

### Filter by primary code

In [19]:
# ph_tx = ph_tx[-ph_tx['Provider Organization Name (Legal Business Name)'].str.contains("NORDSTROM")].reset_index(drop = True)

In [20]:
# ph_tx[ph_tx['taxonomy'].str.contains("261QA0900X")]

In [21]:
# ph_tx.shape

### Filter by name

In [226]:
name_list = ph_tx[ph_tx['Provider Organization Name (Legal Business Name)'].str.contains("SOUND|SHOE|LINGERIE|HAIR")]

In [223]:
name_list[name_list['Provider Organization Name (Legal Business Name)'].str.contains("ORTHO|PROST")]

,NPI,Entity Type Code,Provider Organization Name (Legal Business Name),Provider First Line Business Practice Location Address,City,State2,Provider Business Practice Location Address Postal Code,Provider Business Practice Location Address Telephone Number,Zip5,Address,taxonomy
119,1982903720,2,"JACKRABBIT SHOES AND ORTHOTICS, INC.",4576 E 2ND ST,BENICIA,CA,94510,7077511630,94510,4576 E 2ND ST,335E00000X
154,1609135029,2,M&G CUSTOM SHOES AND ORTHOTICS LLC,2950 BUSKIRK AVE,WALNUT CREEK,CA,94597,9253051855,94597,2950 BUSKIRK AVE,335E00000X
155,1902076987,2,M&G CUSTOM SHOES AND ORTHOTICS LLC,1504 A ST,ANTIOCH,CA,94509,9253051855,94509,1504 A ST,335E00000X


### Convert to EXCEL

In [230]:
# ph_tx.to_excel('list_caliclinic.xlsx',index=False)

## Geo coding

In [20]:
def split_geo(df, add, city, state, zipcode, chunk_size=500):
    df_new = df.copy()
    df_new.reset_index(inplace=True)
    splits = np.ceil(df_new.shape[0]/chunk_size)
    chunk_li = np.array_split(df_new['index'], splits)
    res_li = []
    pick_fi = []
    for i,c in enumerate(chunk_li):
        # Grab data, export to csv
        sub_data = df_new.loc[c, ['index',add,city,state,zipcode]]
        sub_data.to_csv('temp_geo.csv',header=False,index=False)
        # Geo the results and turn back into df
        print(f'Geocoding round {int(i)+1} of {int(splits)}, {datetime.now()}')
        result = cg.addressbatch('temp_geo.csv') #should try/except?
        # May want to dump the intermediate results
        #pi_str = f'pickres_{int(i)}.p'
        #pickle.dump( favorite_color, open( pi_str, "wb" ) )
        #pick_fi.append(pi_str.copy())
        names = list(result[0].keys())
        res_zl = []
        for r in result:
            res_zl.append(list(r.values()))
        res_df = pd.DataFrame(res_zl, columns=names)
        res_li.append(res_df.copy())
        # time.sleep(10) #sleep 10 seconds to not get cutoff from request
    final_df = pd.concat(res_li)
    final_df.rename(columns={'id':'row'}, inplace=True)
    final_df.reset_index(inplace=True, drop=True)
    # Clean up csv file
    os.remove('temp_geo.csv')
    return final_df

In [21]:
geo_pharm = split_geo(ph_tx, add='Address', city='City', state='State2', zipcode='Zip5', chunk_size=500)

Geocoding round 1 of 2, 2023-04-03 05:10:28.155673
Geocoding round 2 of 2, 2023-04-03 05:10:35.946248


In [22]:
print(geo_pharm['match'].value_counts())

True     740
False     68
Name: match, dtype: int64


In [23]:
geo_pharm['rowN'] = geo_pharm['row'].astype(int)
gp2 = geo_pharm.sort_values(by='rowN').reset_index(drop=True)

In [24]:
kg = ['address','match','lat','lon']
kd = ['NPI',
      'Provider Organization Name (Legal Business Name)',
      'Provider Business Practice Location Address Telephone Number',
      'City','State2','Zip5']
final_pharm = pd.concat([ph_tx[kd], gp2[kg]], axis=1)

final_pharm.rename(columns={'Provider Organization Name (Legal Business Name)':'Name',
                      'Provider Business Practice Location Address Telephone Number':'Telephone'}, inplace=True)
final_pharm

,NPI,Name,Telephone,City,State2,Zip5,address,match,lat,lon
0,1548468614,DIMENSION PROSTHETICS & ORTHOTICS,5103243400,UNION CITY,CA,94587,"33374 DOWE AVE, UNION CITY, CA, 94587",True,37.601059,-122.043522
1,1952507303,"SOUND BALANCE AUDIOLOGY, INC",7607217417,OCEANSIDE,CA,92054,"2420 VISTA WAY, OCEANSIDE, CA, 92054",True,33.183241,-117.334400
2,1609934660,COLLIER REHABILITATION SYSTEMS,9259431119,PLEASANT HILL,CA,94523,"3161 PUTNAM BLVD, PLEASANT HILL, CA, 94523",True,37.933074,-122.073511
3,1518006741,"NEW DAY'S DAWN, INC.",6195964042,SANTEE,CA,92071,"10159 MISSION GORGE RD, SANTEE, CA, 92071",True,32.838527,-116.976276
4,1902863202,SUPER CARE INC,6268542283,CITY OF INDUSTRY,CA,91744,"16017 VALLEY BLVD, CITY OF INDUSTRY, CA, 91744",False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
803,1902852056,"ACTIVE LIFE, LLC",8188359441,NORTHRIDGE,CA,91325,"18433 ROSCOE BLVD, NORTHRIDGE, CA, 91325",True,34.220760,-118.534506
804,1942865605,"ACTIVE LIFE, LLC",7605156311,APPLE VALLEY,CA,92307,"16008 KAMANA RD, APPLE VALLEY, CA, 92307",True,34.542401,-117.271759
805,1952609661,"ACTIVE LIFE, LLC",6194886196,SAN DIEGO,CA,92123,"7910 FROST ST, SAN DIEGO, CA, 92123",True,32.800261,-117.154388
806,1952765711,"ACTIVE LIFE, LLC",3233528319,LOS ANGELES,CA,90033,"1700 E CESAR E CHAVEZ AVE, LOS ANGELES, CA, 90033",True,34.051144,-118.217488


In [25]:
#final_pharm.to_csv('clinics_Cali.csv',index=False)

In [26]:
hosp_data = pd.read_csv('clinics_Cali.csv')
hosp_data = hosp_data[hosp_data['match']].copy()
hosp_data.reset_index(inplace=True, drop=True)
hosp_data.head(2)

,NPI,Name,Telephone,City,State2,Zip5,address,match,lat,lon
0,1548468614,DIMENSION PROSTHETICS & ORTHOTICS,5103243400,UNION CITY,CA,94587,"33374 DOWE AVE, UNION CITY, CA, 94587",True,37.601059,-122.043522
1,1952507303,"SOUND BALANCE AUDIOLOGY, INC",7607217417,OCEANSIDE,CA,92054,"2420 VISTA WAY, OCEANSIDE, CA, 92054",True,33.183241,-117.334400


In [27]:
hosp_geo = gpd.GeoDataFrame(hosp_data, geometry=gpd.points_from_xy(hosp_data.lon, hosp_data.lat), crs="EPSG:4326")

In [28]:
cali_counties = gpd.read_file(r'tl_2016_06_cousub/tl_2016_06_cousub.shp')
cali_outline = cali_counties.dissolve('STATEFP')
cali_proj = cali_outline.to_crs('EPSG:5070')
print(cali_outline.crs)

EPSG:4269


In [29]:
def dissolve_buff(point_df,d,resolution):
    bu = point_df.buffer(d,resolution)
    geodf = gpd.GeoDataFrame(geometry=bu)
    geodf['Const'] = 0
    single = geodf.dissolve('Const')
    return single[['geometry']]

In [30]:
def dist_cont(point_df,dist_list,outside,buff_res):
    if point_df.crs != outside.crs:
        print('Point df and Outside df are not the same CRS')
        return None
    # Making outside area out dissolved object
    out_cop = outside[['geometry']].copy()
    out_cop['Constant'] = 1
    out_cop = out_cop.dissolve('Constant')
    # Make sure points are inside area
    inside = point_df.within(out_cop['geometry'][1])
    point_cop = point_df[inside].copy()
    point_cop = point_df.copy()
    point_cop['Constant'] = 1 #Constant for dissolve
    point_cop = point_cop[['Constant','geometry']].copy()
    res_buffers = []
    for i,d in enumerate(dist_list):
        print(f'Doing buffer {d}')
        if i == 0:
            res = dissolve_buff(point_cop, d, buff_res)
            res_buffers.append(res.copy())
        else:
            res_new = dissolve_buff(point_cop, d, buff_res)
            res_buffonly = gpd.overlay(res_new, res, how='difference')
            res = res_new.copy()
            res_buffers.append( res_buffonly.copy() )
    # Now take the difference with the larger area
    print('Working on leftover difference now')
    leftover = gpd.overlay(out_cop, res, how='difference')
    res_buffers.append(leftover)
    for i,d in enumerate(dist_list):
        res_buffers[i]['Distance'] = str(d)
    res_buffers[-1]['Distance'] = 'Outside'
    # New geopandas DF
    comb_df = pd.concat(res_buffers)
    comb_df.reset_index(inplace=True, drop=True)
    return comb_df

In [31]:
hos_proj = hosp_geo.to_crs('EPSG:5070') #'epsg:4269'

dist_met = [2000, 4000, 8000, 16000] #, 32000
buff_city = dist_cont(hos_proj, dist_met, cali_proj, buff_res=100)

Doing buffer 2000
Doing buffer 4000
Doing buffer 8000
Doing buffer 16000
Working on leftover difference now


In [32]:
#Now making folium plot
buff_map = buff_city.to_crs('EPSG:4326')
kv = list(hosp_geo)[1:10]


In [33]:
#"fill": "#00aa22",
#"fill-opacity": 0.5

cols = ['#f1eef6',
'#d7b5d8',
'#df65b0',
'#dd1c77',
'#980043']

buff_map['fill'] = cols
buff_map['fill-opacity'] = 0.35

#os.chdir(r'D:\Dropbox\Dropbox\PublicCode_Git\Blog_Code')

buff_map.to_file('Buffers.geojson', driver='GeoJSON')
hosp_geo.to_file('Hosp.geojson', driver='GeoJSON')